# Lecture 3.2 - Numerical computing and data science - numpy, scipy, pandas
Numpy is the foundation for numerical computing in python. It enables fast numerical operations, by moving computationally expensive computations to faster code.

Numpy itself mainly provides "low-level" functions for handling vector data:
- multi-dimensional numerical arrays to represent vectors, matrices etc (`np.ndarray`)
- multi-dimensional indexing and slicing
- boolean indexing
- functions for searching and filtering data: min/max, argmax, where (e.g. find pos. values: how many, at which indices, which values?)
- basic statistical functions: np.sum, np.mean, np.std
- linear algebra (dot products, eigenvalues etc), Fourier transforms, interpolation

Numpy forms the basis for a whole eco system of scientific computing:
1. _Scipy_ provides more advanced algorithms. We will cover the following:
    - statistics and statistical tests (mean, std, ttest, anova, non-parametric tests) - Week 4
    - curve fitting and optimization - Week 5
    - frequency analyses - Week 6(?)
2. _Pandas_ provides a way for handling tabular data with named columns and rows, plus the ability to load/save common file formats (Week 3, Week 5)
    
_Numpy for matlab users_:
If you know Matlab, than numpy is conceptually very similar. Here is a great resource for matlab users that want to use numpy: [Numpy for Matlab users](https://numpy.org/doc/stable/user/numpy-for-matlab-users.html).

Last remark - numpy, scipy, and pandas are large packages, with many functions. We will only scratch the surface of what can be done with these.


## Why?
So far, we've dealt with small data sets - what if we have realistic amounts of data?
For instance, it is now easy to record electrophysiological activity for many hours on hundreds of channels. Processing these large volumes of data should not take days!!

However, python is slow for raw number crunching. 

As an example, take a moderately long recording - 10 hours of single channel data voltage recordings:

In [6]:
import numpy as np
data = np.load('data/voltage_trace_long.npz')

Detecting spikes in these data with our "old" python code takes a long time - 17 seconds.

In [7]:
%%time
threshold = 10
spike_times_old = []
spike_voltages_old = []

for v, t in zip(data['voltage_trace'], data['times_seconds']):
    if v > threshold:
        spike_voltages_old.append(v)
        spike_times_old.append(t)
print(f"Detected {len(spike_times_old)} spikes.")

Detected 5270400 spikes.
CPU times: user 10.5 s, sys: 288 ms, total: 10.8 s
Wall time: 10.8 s


The same, implemented in numpy is not only faster - it takes less than two seconds - but also more concise and easier to read!

Below code will make more sense at the end of this lecture.

In [8]:
%%time
idx = data['voltage_trace'] > threshold  # determine which values in voltage_trace exceed the threshold
spike_times = data['times_seconds'][idx]  # take the time of the super-threshold values
spike_voltages = data['voltage_trace'][idx]  # take the voltage of the super-threshold values
print(f"Detected {len(spike_times)}")

Detected 5270400
CPU times: user 1.64 s, sys: 294 ms, total: 1.93 s
Wall time: 1.94 s


## From lists to numpy arrays
We have a list of numbers and want to subtract the mean from each value in the list.
This is how it looks using core python:

In [11]:
data = [24, 6, 32, 9, 32, 43]

# compute the mean
mean = sum(data) / len(data)

# subtract the mean from each element of the list
new_data = []
for d in data:
    new_data.append(d - mean)

Wouldn't it be nice if we could write code that is closer to how we would express the mathematical operation we want to perform, without needing a for loop?

Like, $\vec{y} = \vec{x} - m(\vec{x})$, where $m$ is a mathematical function that returns the mean of the elements in $\vec{x}$.

You can't do this with python lists, because python lists are a generic data type, not optimized for numerical data and computation.

This is exactly where numpy comes into play - it makes working with numerical data easier and faster, by providing a specialized, numerical, data type - the numpy array - and numerical functions:

In [12]:
import numpy as np  # this is the standard way of importing numpy, as np!

array = np.array(data)  # cast the list into a numpy array
new_array = array - np.mean(data)  # compute the mean and subtract it from each element in the array
new_array

array([ -0.33333333, -18.33333333,   7.66666667, -15.33333333,
         7.66666667,  18.66666667])

So, while you can express any computation using core python, often with for loops, numpy allows you to express these computations more concisely, without for loops.

In addition, numpy (and scipy) provide functions for many complex computations, so you don't have to implement them yourself.

Overall, this makes code easier to read (less text) and faster to execute.

We will see more examples of this below.

### Numpy arrays
A numpy array can be created from a python list using `np.array`.

We can inspect the array:

In [2]:
import numpy as np

data = np.array([24, 6, 32,9, 32, 43])
print(f"{data=}")
print(f"Number of dimensions (1D, 2D, ND): {data.ndim=}")
print(f"Shape: {data.shape=}")
print(f"This returns the size of the first dimension: {len(data)=}")
print(f"Data type of the array elements {data.dtype=}")

data=array([24,  6, 32,  9, 32, 43])
Number of dimensions (1D, 2D, ND): data.ndim=1
Shape: data.shape=(6,)
This returns the size of the first dimension: len(data)=6
Data type of the array elements data.dtype=dtype('int64')


Numpy arrays can be multi-dimensional. This is useful for a lot of scientific data you might encounter:
- black and white images are 2D matrices of pixel color values - XxY
- color images are 3D matrices of pixel color values - XxYxColor (Color=RGB)
- volumetric data (microscopy stacks, structural MRI stacks) are 3D matrices - stacks of images - XxYxZ
- behavioral videos or fMRI data can be represented as 3D matrices - a time series of images - XxYxTime
- multi-channel audio or electrophysiology data are often 2D matrices - TimexChannel

The following will create a matrix (2D array) from a nested list:

In [4]:
data = np.array([[24, 6, 32],[9, 32, 43]])
print(f"{data=}")
print(f"Number of dimensions (1D, 2D, ND): {data.ndim=}")
print(f"Shape: {data.shape=}")
print(f"This returns the size of the first dimension: {len(data)=}")
print(f"Data type of the array elements {data.dtype=}")

data=array([[24,  6, 32],
       [ 9, 32, 43]])
Number of dimensions (1D, 2D, ND): data.ndim=2
Shape: data.shape=(2, 3)
This returns the size of the first dimension: len(data)=2
Data type of the array elements data.dtype=dtype('int64')


### Array creation

Numpy also provides tools for creating "empty" arrays of a specific size, to be filled with data later:

In [5]:
print('ones:', np.ones(shape=(4,)))
print('zeros:', np.zeros(shape=(4,)))
print('ones 2D:', np.ones(shape=(2,4)))
print('ones, with same dtype and shape as the "data" array:', np.ones_like(data))
print('arange:', np.arange(-1, 1, 0.25))  # like range but allows non-integer arguments (start, stop, step)
print('linspace:', np.linspace(0, 10, 4))

ones: [1. 1. 1. 1.]
zeros: [0. 0. 0. 0.]
ones 2D: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
ones, with same dtype and shape as the "data" array: [[1 1 1]
 [1 1 1]]
arange: [-1.   -0.75 -0.5  -0.25  0.    0.25  0.5   0.75]
linspace: [ 0.          3.33333333  6.66666667 10.        ]


### Indexing and slicing
Indexing and slicing numpy arrays works similar as for lists.

However, numpy also provides multi-dimensional indexing:

In [6]:
data = np.array([[24, 6, 32],[9, 32, 43]])

print(data)
print(f"first row: {data[0, :]=}")
print(f"element at second row, first column: {data[1, 0]=}")
print(f"same using list-style indexing: {data[1][0]=}")
print(f"first column: {data[:, 0]=}")
print(f"first two columns: {data[:, :2]=}")
print(f"last column: {data[:, -1]=}")

[[24  6 32]
 [ 9 32 43]]
first row: data[0, :]=array([24,  6, 32])
element at second row, first column: data[1, 0]=9
same using list-style indexing: data[1][0]=9
first column: data[:, 0]=array([24,  9])
first two columns: data[:, :2]=array([[24,  6],
       [ 9, 32]])
last column: data[:, -1]=array([32, 43])


Assignment works similarly:

In [7]:
d = np.zeros((10, 10))
print(d)

d[1:4, 5:8] = 100
print(d)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100. 100. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100. 100. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100. 100. 100.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


One can even use arrays to index other arrays or lists:

In [9]:
data = np.arange(0, 40, 2)

indices = data > 10
print(f"{data=}")
print(f"{indices=}")
print(f"data values at the indices: {data[indices]=}")

data=array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38])
indices=array([False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])
data values at the indices: data[indices]=array([12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38])


__Important__, the type of the list elements we want to use for indexing needs to be int, not float.

In [10]:
indices = np.array([1.0, 7.0, 18.0])
print(f"Indices are of type float: {indices.dtype=}")
print(f"This does not work: {data[indices]=}")

Indices are of type float: indices.dtype=dtype('float64')


IndexError: arrays used as indices must be of integer (or boolean) type

### Array types and casting
The type of all data in an array can be easily changed. This is useful to ensure you can use an array for indexing!

In [14]:
x = np.arange(-4, 4, 0.5)
print(f"{x=}")
print(f"{x.dtype=}")  # default type if float64
print(f"{x.astype(int)=}")  # cast to int
print(f"{x.astype(np.uint)=}")  # cast to unsigned int

x=array([-4. , -3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,
        1.5,  2. ,  2.5,  3. ,  3.5])
x.dtype=dtype('float64')
x.astype(int)=array([-4, -3, -3, -2, -2, -1, -1,  0,  0,  0,  1,  1,  2,  2,  3,  3])
x.astype(np.uint)=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3], dtype=uint64)


/var/folders/bc/5m_c7nkj1vnc2w7xmmnhpfww0000gn/T/ipykernel_30083/2679548855.py:5: RuntimeWarning: invalid value encountered in cast
  print(f"{x.astype(np.uint)=}")  # cast to unsigned int


### Math

In [15]:
print(f"{data=}")
print(f"{data + 10=}")
print(f"{data - np.ones_like(data) * 2=}")
print(f"{data - np.array([[1], [2.5]])=}")
print(f"{data * np.array([[1], [2.5]])=}")

data=array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38])
data + 10=array([10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42,
       44, 46, 48])
data - np.ones_like(data) * 2=array([-2,  0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30,
       32, 34, 36])
data - np.array([[1], [2.5]])=array([[-1. ,  1. ,  3. ,  5. ,  7. ,  9. , 11. , 13. , 15. , 17. , 19. ,
        21. , 23. , 25. , 27. , 29. , 31. , 33. , 35. , 37. ],
       [-2.5, -0.5,  1.5,  3.5,  5.5,  7.5,  9.5, 11.5, 13.5, 15.5, 17.5,
        19.5, 21.5, 23.5, 25.5, 27.5, 29.5, 31.5, 33.5, 35.5]])
data * np.array([[1], [2.5]])=array([[ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24.,
        26., 28., 30., 32., 34., 36., 38.],
       [ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.,
        65., 70., 75., 80., 85., 90., 95.]])


more functions:

In [16]:
data = np.linspace(0, 1, 10)
print(f"{data=}")
print(f"{np.mean(data)=}")
print(f"{np.prod(data)=}")
print(f"{np.diff(data)=}")

data=array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])
np.mean(data)=0.5
np.prod(data)=0.0
np.diff(data)=array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111, 0.11111111])


In many numpy functions, the `axis` argument allows you to apply an operation only to an axis, not to the full matrix.
This allows you to compute the mean only over rows, or only over columns.

In [17]:
data = np.array([[1, 2, 3], [10, 20, 30]])
print(f"{data=}")
print(f"{np.mean(data, axis=0)=}")
print(f"{np.mean(data, axis=1)=}")

data=array([[ 1,  2,  3],
       [10, 20, 30]])
np.mean(data, axis=0)=array([ 5.5, 11. , 16.5])
np.mean(data, axis=1)=array([ 2., 20.])


### Searching arrays and Boolean indexing
Just like with math, we can also apply comparisons to all elements of an array. This creates boolean arrays:

In [18]:
data = np.array([[24, 6, 30],[9, 32, 43]])
print(f"{data=}")
print(f"{data > 10=}")

data=array([[24,  6, 30],
       [ 9, 32, 43]])
data > 10=array([[ True, False,  True],
       [False,  True,  True]])


We can then use the boolean array as an index. This will return only the data values for which the boolean index is `True`:

In [19]:
print(f"{data=}")
print(f"{data[data > 10]}")
print(f"{data[data < 10]}")

data=array([[24,  6, 30],
       [ 9, 32, 43]])
[24 30 32 43]
[6 9]


`np.where` will return the indices at which the expression is True:

In [20]:
print(f"{np.where(data < 10)}")

(array([0, 1]), array([1, 0]))


To combine different conditions, we need to logically combine the indices, using `np.logical_and` or `np.logical_or`:

In [21]:
lower = data > 10
upper = data < 31
both = np.logical_and(lower, upper)  # takes two boolean arrays, returns a new array in which only elements are True that where True in the two inputs - lower AND upper

print(data)
print(lower)
print(upper)
print(both)
print(data[both])

[[24  6 30]
 [ 9 32 43]]
[[ True False  True]
 [False  True  True]]
[[ True  True  True]
 [ True False False]]
[[ True False  True]
 [False False False]]
[24 30]


In [22]:
lower = data < 10
upper = data > 31
both = np.logical_or(lower, upper)  # takes two boolean arrays, returns a new array in which all elements are True that where True in at least one of the two inputs - lower OR upper

print(data)
print(lower)
print(upper)
print(both)
print(data[both])

[[24  6 30]
 [ 9 32 43]]
[[False  True False]
 [ True False False]]
[[False False False]
 [False  True  True]]
[[False  True False]
 [ True  True  True]]
[ 6  9 32 43]


In [23]:
print(f"{data=}")
print(f"{np.max(data)=}")  # max over all axes
print(f"{np.min(data)=}")  # min over all axes
print(f"{np.min(data, axis=0)=}")  # min for each column
print(f"{np.argmin(data)=}")    # index of the smallest value, after "flattening" the matrix
print(f"{data.flatten()[1]=}")
print(f"{np.max(data, axis=1)=}")
rows, cols = np.where(data > 10)
print(f"{rows, cols=}")

data=array([[24,  6, 30],
       [ 9, 32, 43]])
np.max(data)=43
np.min(data)=6
np.min(data, axis=0)=array([ 9,  6, 30])
np.argmin(data)=1
data.flatten()[1]=6
np.max(data, axis=1)=array([30, 43])
rows, cols=(array([0, 0, 1, 1]), array([0, 2, 1, 2]))
